In [ ]:
#숫자 6자리로 Json과 Jpeg 매칭
import os
import shutil
import re
import json

# 경로 설정
img_dir = r"C:\Users\User\Downloads\095.만화·웹툰 생성 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS_07. 일상"
label_dir = r"C:\Users\User\Downloads\095.만화·웹툰 생성 데이터\3.개방데이터\1.데이터\Training\02.라벨링데이터\TL_07. 일상"
select_label_dir = r'C:\Users\User\select_label2'

# select_label 폴더가 없으면 생성
os.makedirs(select_label_dir, exist_ok=True)

# 이미지 파일 이름에서 숫자 추출하는 함수
def extract_number_from_filename(filename):
    # 파일명에서 숫자만 추출
    match = re.search(r'\d+', filename)
    if match:
        return match.group(0)
    return None

# JSON 파일에 'context' 키가 없는지 확인하는 함수
def check_no_context(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    directing = data.get('label', {}).get('directing', {})
    return 'context' not in directing or not directing['context']


# 이미지 파일 목록 가져오기
img_files = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpeg', '.JPEG'))]

# 라벨 파일 목록 가져오기
label_files = [f for f in os.listdir(label_dir) if f.lower().endswith('.json')]

# 이미지 파일 이름에서 숫자를 추출하여 해당 번호와 일치하는 JSON 파일을 select_label 폴더로 이동
for img_file in img_files:
    img_number = extract_number_from_filename(img_file)
    if img_number:
        for label_file in label_files:
            label_number = extract_number_from_filename(label_file)
            if img_number == label_number:
                # 일치하는 JSON 파일의 경로
                label_file_path = os.path.join(label_dir, label_file)

                # JSON 파일에 'context' 키가 없는 경우에만 이동
                if check_no_context(label_file_path):
                    shutil.move(label_file_path, os.path.join(select_label_dir, label_file))
                    print(f"Moved: {label_file}")

print("파일 이동 완료!")

In [ ]:
#짝 맞는 Jpeg가 존재하지 않을경우 Json 파일 삭제제
import os

# 폴더 경로 설정
img_folder = r'C:\Users\User\Desktop\Img'
json_folder = r'C:\Users\User\Desktop\Json'

# 파일명에서 확장자 앞 6자리 추출
def extract_suffix(filename):
    return filename[-11:-4]  # 확장자 앞 6자리 추출

# Json 폴더에서 숫자를 추출하여 집합으로 저장
json_suffixes = set()
for json_file in os.listdir(json_folder):
    if json_file.endswith('.json'):
        json_suffix = extract_suffix(json_file)  # 확장자 앞 6자리 추출
        json_suffixes.add(json_suffix)

# Img 폴더에서 JPEG 파일 처리
for img_file in os.listdir(img_folder):
    if img_file.endswith('.JPEG'):
        img_suffix = extract_suffix(img_file)  # 확장자 앞 6자리 추출
        if img_suffix not in json_suffixes:
            # Json 폴더에 없는 경우 삭제
            img_path = os.path.join(img_folder, img_file)
            try:
                os.remove(img_path)
                print(f"삭제됨: {img_file}")
            except Exception as e:
                print(f"파일 삭제 중 오류 발생: {img_file}. 오류: {str(e)}")

print("작업 완료")

In [ ]:
#중복 이미지 삭제
import os
import shutil

# 폴더 경로 설정
desktop_folder = r'C:\Users\User\Desktop\Img'
source_folder = r'C:\Users\User\Downloads\095.만화·웹툰 생성 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS_07. 일상'

# Desktop\Img 폴더의 JPEG 파일 목록 가져오기
desktop_files = [f.lower() for f in os.listdir(desktop_folder) if f.lower().endswith('.jpeg')]

# 소스 폴더의 파일 순회
for filename in os.listdir(source_folder):
    if filename.lower() in desktop_files:
        file_path = os.path.join(source_folder, filename)
        try:
            os.remove(file_path)
            print(f"삭제됨: {filename}")
        except Exception as e:
            print(f"파일 삭제 중 오류 발생: {filename}. 오류: {str(e)}")

print("작업 완료")

In [ ]:
#번역
import os
import json
from googletrans import Translator
import time

# 번역기 초기화 함수
def translate_with_retry(text, dest='en', src='auto', max_retries=3, delay=5):
    translator = Translator()
    translator.raise_Exception = True
    for attempt in range(max_retries):
        try:
            return translator.translate(text, dest=dest, src=src).text
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(delay)
            else:
                print(f"텍스트 번역 오류: {text}. 오류: {str(e)}")
                return text  # 모든 재시도가 실패하면 원본 텍스트 반환

# 폴더 경로
folder_path = r"C:\Users\User\select_label2"

# 폴더 내 JSON 파일 처리
file_list = [file for file in os.listdir(folder_path) if file.endswith('.json')]
total_files = len(file_list)  # 총 파일 수

for idx, file_name in enumerate(file_list, start=1):
    file_path = os.path.join(folder_path, file_name)

    # 진행 상황 출력
    print(f"Processing file {idx}/{total_files}: {file_name}")

    # JSON 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 번역 수행
    # 1. 'prompt' 번역 (라벨 안에 존재)
    if 'label' in data and 'prompt' in data['label'] and data['label']['prompt']:
        data['label']['prompt'] = translate_with_retry(data['label']['prompt'])

    # 2. 'caption' 번역
    if 'caption' in data and data['caption']:
        data['caption'] = translate_with_retry(data['caption'])

    # 변경된 JSON 저장
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
#prompt 뒤 3개의 단어가 불필요해 삭제하는 코드
import os
import json

def clean_prompt(prompt):
    # 쉼표로 분리
    words = prompt.split(',')

    # 뒤에서 세 번째 쉼표까지의 내용만 유지
    cleaned_words = words[:-3] if len(words) > 3 else words

    # 선택된 단어들을 다시 쉼표와 공백으로 연결
    return ', '.join(word.strip() for word in cleaned_words)

# JSON 파일들이 있는 폴더 경로 정의
folder_path = r'C:\Users\User\select_label2'  # 실제 폴더 경로로 변경해주세요

# 폴더 내 각 파일을 순회
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # JSON 파일 열기 및 로드
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # 'prompt' 필드 수정
        if 'prompt' in data['label']:
            data['label']['prompt'] = clean_prompt(data['label']['prompt'])

        # 수정된 JSON을 파일에 다시 저장
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

print("모든 JSON 파일의 prompt 필드가 성공적으로 업데이트되었습니다.")

In [ ]:
#말풍선이 존재할 경우 prompt 앞에 bubble 단어 추가하기
import os
import json

def add_bubble_to_prompt(data):
    context = data['label']['directing'].get('context', [])

    has_bubble = any('bubble' in item for item in context)

    if has_bubble:
        prompt = data['label']['prompt']
        data['label']['prompt'] = f"bubble, {prompt}"

    return data

folder_path = r"C:\Users\User\DataSet\Json"

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)

        modified_data = add_bubble_to_prompt(json_data)

        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(modified_data, file, ensure_ascii=False, indent=2)

        print(f"Processed and saved: {filename}")

print("All JSON files have been processed and saved.")